In [ ]:
!pip install mne
# Install required Python libraries for EEG processing and machine learning
!pip install numpy scipy matplotlib latexify-py skfeature-chappers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 66.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.3/66.3 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 10.6 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import mne
import numpy as np
import matplotlib.pyplot as plt
import warnings

# Ignore warnings for a cleaner output
warnings.filterwarnings("ignore", message=".*annotation.*")

In [ ]:
folder_path = "/content/drive/MyDrive/SSVEP-based-EEG-signal-processing-main/SSVEP-based-EEG-signal-processing-main/Data"

def data_path(folder_path, data_format="gdf"):
    path_files = []  # Store paths of matching files
    files = []  # Store file names
    folders = []  # Store folder names

    # Walk through the directory and collect relevant files/folders
    for root, dirnames, filenames in os.walk(folder_path):
        for filename in filenames:
            if filename.endswith(f".{data_format}"):
                full_path = os.path.join(root, filename)
                path_files.append(full_path)
                files.append(filename)
        folders.extend(dirnames)

    return path_files, files, folders

# Get the paths of all GDF files
path_files, files, folders = data_path(folder_path, data_format="gdf")

# Print the collected file paths
print("Found files:", files)
print("Full paths:", path_files)

Found files: ['Subject4-[2012.04.08-16.06.48].gdf', 'Subject3-[2012.04.07-18.27.18].gdf', 'Subject2-[2012.04.07-19.44.23].gdf', 'Subject3-[2012.04.07-18.45.34].gdf', 'Subject3-[2012.04.07-18.17.50].gdf', 'Subject2-[2012.04.07-19.36.29].gdf', 'Subject2-[2012.04.07-19.57.52].gdf', 'Subject3-[2012.04.07-18.53.10].gdf', 'Subject2-[2012.04.07-19.27.02].gdf', 'Subject4-[2012.04.08-16.27.27].gdf', 'Subject5-[2012.04.09-19.38.11].gdf', 'Subject5-[2012.04.09-19.48.56].gdf', 'Subject5-[2012.04.09-20.02.48].gdf', 'Subject5-[2012.04.09-19.56.38].gdf', 'Subject4-[2012.04.08-16.19.25].gdf', 'Subject4-[2012.04.08-16.35.27].gdf']
Full paths: ['/content/drive/MyDrive/SSVEP-based-EEG-signal-processing-main/SSVEP-based-EEG-signal-processing-main/Data/Subject4-[2012.04.08-16.06.48].gdf', '/content/drive/MyDrive/SSVEP-based-EEG-signal-processing-main/SSVEP-based-EEG-signal-processing-main/Data/Subject3-[2012.04.07-18.27.18].gdf', '/content/drive/MyDrive/SSVEP-based-EEG-signal-processing-main/SSVEP-based-EE

In [ ]:
# Ensure the path_files variable contains GDF file paths
print(f"Using GDF file: {path_files[0]}")

# Read the GDF file into a raw MNE object
raw = mne.io.read_raw_gdf(path_files[0], verbose=0)

# Extract the channel names from the raw data
channels_name = raw.ch_names

# Get the EEG data and transpose it to have channels as rows and samples as columns
data = 1e6 * raw.get_data().T  # Convert to microvolts if necessary

# Get the sampling frequency of the EEG data
fs = raw.info['sfreq']

# Extract labels from annotations
labels = raw.annotations.description

# Get the events and their corresponding indices
events, event_ind = mne.events_from_annotations(raw, verbose=0)

# Print all the relevant information
print(f"Data: {data.shape} \n")
print(f"Channels Name: {channels_name} \n")
print(f"Labels: {labels} \n")
print(f"Events: {events} \n")
print(f"Event Indices: {event_ind} \n")

Using GDF file: /content/drive/MyDrive/SSVEP-based-EEG-signal-processing-main/SSVEP-based-EEG-signal-processing-main/Data/Subject4-[2012.04.08-16.06.48].gdf
Data: (127200, 8) 

Channels Name: ['Oz', 'O1', 'O2', 'PO3', 'POz', 'PO7', 'PO8', 'PO4'] 

Labels: ['32769' '33024' '32779' '32780' '33026' '32779' '32780' '33027' '32779'
 '32780' '33025' '32779' '32780' '33026' '32779' '32780' '33025' '32779'
 '32780' '33024' '32779' '32780' '33027' '32779' '32780' '33025' '32779'
 '32780' '33026' '32779' '32780' '33027' '32779' '32780' '33024' '32779'
 '32780' '33026' '32779' '32780' '33024' '32779' '32780' '33027' '32779'
 '32780' '33025' '32779' '32780' '33024' '32779' '32780' '33027' '32779'
 '32780' '33025' '32779' '32780' '33026' '32779' '32780' '33027' '32779'
 '32780' '33024' '32779' '32780' '33025' '32779' '32780' '33026' '32779'
 '32780' '33025' '32779' '32780' '33027' '32779' '32780' '33026' '32779'
 '32780' '33024' '32779' '32780' '33027' '32779' '32780' '33026' '32779'
 '32780' '3302

In [ ]:
import pandas as pd
# Create a DataFrame for the EEG data with channel names as columns
df = pd.DataFrame(data, columns=channels_name)
# Add the labels to the DataFrame as a new column
df['Labels'] = labels
# Display the first few rows of the DataFrame
print(df.head())

Filtering/ feature Extraction / Preprocessing

In [ ]:
#Import required libraries
import numpy as np
import mne
import warnings
import sys

sys.path.append('/content/drive/MyDrive/SSVEP-based-EEG-signal-processing-main/SSVEP-based-EEG-signal-processing-main/Code/Python/Functions')

# Import functions from the .py files in the Functions folder
from Filtering import filtering

# Step 4: Define your parameters for filtering
trial = 8            # Define trial number (trial 1 in Python index starts from 0)
order = 3            # Define filter order
f_low = 0.05         # Define lower cutoff frequency for the bandpass filter (Hz)
f_high = 100         # Define upper cutoff frequency for the bandpass filter (Hz)
notch_freq = 50      # Define frequency to be removed from the signal for notch filter (Hz)
quality_factor = 20  # Define quality factor for the notch filter
notch_filter = "on"  # on or off
filter_active = "on" # on or off
design_method = "IIR" # IIR or FIR
type_filter = "bandpass"  # low, high, bandpass, or bandstop
freq_stim = 13       # Define stimulation frequency


# Step 5: Apply bandpass filtering to the EEG data
filtered_data = filtering(data1, f_low, f_high, order, fs, notch_freq, quality_factor,
                          filter_active, notch_filter, type_filter, design_method)

# Print the shape of the filtered data to verify
print(f"Filtered data shape: {filtered_data.shape}")


Data shape : (1280, 8, 160)
Filtered data shape: (1280, 8, 160)


CAR Filter (Common average reference)

In [ ]:
#Import the CAR filter function
from Common_average_reference import car

# Step 3: Apply CAR filter to the data
# filtered_data shape: (1280, 8, 160)
data_car = car(filtered_data, reference_channel=None)  # Use all channels for average reference

# Step 4: Verify the shape of CAR-filtered data
print(f"CAR-filtered data shape: {data_car.shape}")

# Step 5: Extract the trial-specific data if needed
trial = 0  # Define trial number (0-indexed)
trial_data_car = data_car[:, :, trial]  # Extract data for the selected trial

# Step 6: Verify the trial data shape
print(f"CAR-filtered data for Trial {trial + 1}: {trial_data_car.shape}")

CAR-filtered data shape: (1280, 8, 160)
CAR-filtered data for Trial 1: (1280, 8)


CCA

In [ ]:
# Import functions
from Filtering import filtering
from Common_average_reference import car
from CCA import cca

# ------------------------------------ Step 2: Load and Combine Data ----------------------------------------
# Assuming data1, data2, and data3 are already loaded
data_total = np.concatenate((data1, data2, data3), axis=2)

# Generate labels for each dataset (0, 1, 2)
labels = np.concatenate((np.full(data1.shape[-1], 0),
                         np.full(data2.shape[-1], 1),
                         np.full(data3.shape[-1], 2)))

# ------------------------------------ Step 3: Filter the Data ----------------------------------------
order = 4
f_low = 0.05
f_high = 100
notch_freq = 50
quality_factor = 20
notch_filter = "on"
filter_active = "off"
type_filter = "bandpass"

filtered_data = filtering(data_total, f_low, f_high, order, fs,
                          notch_freq, quality_factor, filter_active,
                          notch_filter, type_filter)

print(f"Filtered Data Shape: {filtered_data.shape}")

# ----------------------------------- Step 4: Apply CAR Filter ----------------------------------------
data_car = car(filtered_data)
print(f"CAR-Filtered Data Shape: {data_car.shape}")

# ----------------------------------- Step 5: Perform CCA Classification ----------------------------------------
num_channel = [0, 1, 2]   # List of channels to use
num_harmonic = 4          # Number of harmonics
f_stim = [13, 21, 17]     # Frequencies used for stimulation

# Use your CCA function to classify the EEG signals
predict_label = cca(data_car, fs, f_stim, num_channel, num_harmonic)

print(predict_label)

# ------------------------------------ Step 6: Calculate Accuracy ----------------------------------------
accuracy = np.sum(labels == predict_label) / len(predict_label) * 100
print(f"Classification Accuracy: {accuracy:.2f}%")

Filtered Data Shape: (1280, 8, 480)
CAR-Filtered Data Shape: (1280, 8, 480)
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 0. 1. 0. 1.
 0. 0. 1. 0. 1. 1. 1. 1. 2. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1.
 1. 1. 1. 1. 1. 1. 2. 1. 1. 1. 1. 0. 1. 2. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 2. 1. 1. 1. 0. 1.
 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 1. 1. 1. 1. 1.
 2. 1. 1. 1. 1. 1. 1. 1. 1. 2. 1. 2. 1. 1. 0. 0.

Feature Extraction using CCA

In [ ]:
#Import functions from the files
import Filtering
import Common_average_reference
import CCA_Feature_Extraction
import numpy as np

#Combine all datasets
data_total = np.concatenate((data1, data2, data3), axis=2)
labels = np.concatenate((np.full(data1.shape[-1], 0),
                         np.full(data2.shape[-1], 1),
                         np.full(data3.shape[-1], 2)))

# Define filtering parameters
order = 3                # Define filter order
notch_freq = 50          # Define frequency to be removed from the signal for notch filter (Hz)
quality_factor = 20      # Define quality factor for the notch filter
subbands = [[12, 16, 20], [14, 18, 22]]
f_low = np.min(subbands) - 1  # Define lower cutoff frequency for the bandpass filter (Hz)
f_high = np.max(subbands) + 1  # Define upper cutoff frequency for the bandpass filter (Hz)
notch_filter = "on"       # on or off
filter_active = "on"      # on or off
type_filter = "bandpass"  # low, high, bandpass, or bandstop

# Apply notch filter to the EEG data
filtered_data = Filtering.filtering(data_total, f_low, f_high, order, fs,
                                     notch_freq, quality_factor,
                                     filter_active, notch_filter, type_filter)

# Perform Common Average Reference (CAR)
data_car = Common_average_reference.car(filtered_data)

# Define parameters for feature extraction
num_channel = [0, 1, 2]   # List of channels to use
num_harmonic = 2          # Number of harmonics
f_stim = [13, 21, 17]     # Frequencies stimulation

title = f"Feature Extraction using CCA"

# Perform CCA feature extraction
features_extraction = CCA_Feature_Extraction.cca_feature_extraction(data_car, fs, f_stim, num_channel, num_harmonic)

# Print or visualize the extracted features
print("Extracted Features: ", features_extraction)
print("Extracted Features Shape: ", features_extraction)

Extracted Features:  [[8.48613455e-02 1.54841890e-03 1.33326683e-04 ... 1.58246850e-03
  1.06533270e-03 5.81540495e-05]
 [6.66169000e-02 5.70067345e-03 8.63946117e-05 ... 2.63545705e-03
  2.29649073e-04 1.03755950e-04]
 [1.42600851e-02 1.74431970e-04 2.57166498e-05 ... 3.34775379e-04
  7.92737126e-05 3.45241246e-05]
 ...
 [8.77509493e-03 2.36960440e-03 8.25896660e-05 ... 4.97390480e-02
  6.14588787e-03 1.69494663e-04]
 [1.64460169e-03 1.78155409e-04 2.32833164e-05 ... 1.63746702e-02
  1.01678326e-02 1.23096439e-04]
 [3.45830215e-03 1.76787469e-03 5.19363368e-04 ... 4.43240847e-02
  8.52655063e-03 5.35432503e-04]]
Extracted Features Shape:  [[8.48613455e-02 1.54841890e-03 1.33326683e-04 ... 1.58246850e-03
  1.06533270e-03 5.81540495e-05]
 [6.66169000e-02 5.70067345e-03 8.63946117e-05 ... 2.63545705e-03
  2.29649073e-04 1.03755950e-04]
 [1.42600851e-02 1.74431970e-04 2.57166498e-05 ... 3.34775379e-04
  7.92737126e-05 3.45241246e-05]
 ...
 [8.77509493e-03 2.36960440e-03 8.25896660e-05 ...

Feature Selection

In [ ]:
# Import the feature selection function from the uploaded file
from Feature_selections import feature_selecions

# Define parameters for feature selection
num_features = 4
n_neighbors_MI = 5                 # Number of neighbors to consider for mutual information calculation.
L1_Parameter = 0.1                 # Parameter value for L1 regularization.
threshold_var = 0.001              # The threshold used for variance thresholding.
type_feature_selection = "anova"    # Options: var, anova, mi, ufs, rfe, rf, l1fs, tfs, fs, ffs, bfs
title = f"Feature selection using {type_feature_selection}"

# Perform feature selection
features = feature_selecions(features_extraction, labels, num_features, threshold_var,
                              n_neighbors_MI, L1_Parameter, type_feature_selection)

# Display the selected features
print(f"Selected features shape: {features.shape}")

Selected features shape: (480, 4)


Classification Models

In [ ]:
# Step 1: Import necessary libraries
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Assuming `features_extraction` and `labels` are already available from the previous steps

# Step 2: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    features_extraction, labels, test_size=0.2, random_state=42
)
# Step 3: Standardize the features (important for models like SVM)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


# Support Vector Machine
svm_model = SVC(kernel='linear', probability=True, random_state=42)
svm_model.fit(X_train, y_train)

# Step 5: Evaluate models on the test set
models = {#"Logistic Regression": logreg,
           "SVM": svm_model
          #  , "Random Forest": rf_model
          }

for name, model in models.items():
    y_pred = model.predict(X_test)
    print(f"--- {name} ---")
    print(f"Accuracy: {accuracy_score(y_test, y_pred):.2f}")
    print(classification_report(y_test, y_pred))

# Step 6: Define action mapping based on predictions
def action_mapping(prediction):
    actions = {
        0: "13",
        1: "21",
        2: "17"
    }
    return actions.get(prediction, "Unknown Action")

print(f"Xtest = {X_test.shape}")
print(f"Xtrain = {X_train.shape}")


# Step 7: Test the model with some example data
for i in range (features.shape[1]):
    sample = X_test[i].reshape(1, -1)  # Example data point
    predicted_label = rf_model.predict(sample)[0]
    predicted_action = action_mapping(predicted_label)

    print(f"Predicted Action: {predicted_action}")


--- SVM ---
Accuracy: 1.00
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        34
           1       1.00      1.00      1.00        27
           2       1.00      1.00      1.00        35

    accuracy                           1.00        96
   macro avg       1.00      1.00      1.00        96
weighted avg       1.00      1.00      1.00        96

Xtest = (96, 9)
Xtrain = (384, 9)
Predicted Action: 13
Predicted Action: 17
Predicted Action: 17
Predicted Action: 21


## DEPLOYMENT

In [ ]:
import numpy as np

# Load the .npy file
data1 = np.load('/content/drive/MyDrive/SSVEP-based-EEG-signal-processing-main/SSVEP-based-EEG-signal-processing-main/trials/13hz.npy') [:,:,0]
data2 = np.load('/content/drive/MyDrive/SSVEP-based-EEG-signal-processing-main/SSVEP-based-EEG-signal-processing-main/trials/21hz.npy')[:,:,0]
data3 = np.load('/content/drive/MyDrive/SSVEP-based-EEG-signal-processing-main/SSVEP-based-EEG-signal-processing-main/trials/17hz.npy')[:,:,0]

# Example event information (you should have these from the old data)
event_ind = {'32769': 1, '32770': 2, '32779': 3, '32780': 4, '33024': 5,
             '33025': 6, '33026': 7, '33027': 8}

# Original labels (from the old data)
labels_old = ['32769', '33024', '32779', '32780', '33026', '32779', '32780', '33027',
              '32779', '32780', '33025', '32779', '32780', '33026', '32779', '32780', '33025']

# For this example, I assume you're extracting labels based on row indices (adjust based on actual event times)
# This should match the number of rows in your new datasets (1280 rows)
labels_new = []
for i in range(data1.shape[0]):
    event_idx = i % len(labels_old)  # Adjust this if the labels are not sequential
    labels_new.append(labels_old[event_idx])

# Convert list to a NumPy array
labels_new = np.array(labels_new)

# Now, we have the new data and corresponding labels (1280 rows x 8 columns, labels array with 1280 items)
# Data Channels Names
channels_name = ['Oz', 'O1', 'O2', 'PO3', 'POz', 'PO7', 'PO8', 'PO4']  # Example for 8 channels

# Convert data to microvolts (if necessary, depending on your data's scale)
data = 1e6 * data1  # Assuming data1, data2, data3 are the subsets

# Sampling frequency (adjust based on your data)
fs = 256  # Example, adjust as per your dataset's fs

# Placeholder for events (adjust according to your dataset's event times and values)
events = np.array([[44752, 0, 1], [45264, 0, 5], [45520, 0, 3]])  # Example events
event_ind = {'32769': 1, '32770': 2, '32779': 3, '32780': 4, '33024': 5,
             '33025': 6, '33026': 7, '33027': 8}  # Event mapping

# Print all the relevant information for the new data and labels
print(f"Data: {data.shape} \n")
print(f"Channels Name: {channels_name} \n")
print(f"Labels: {labels_new} \n")
print(f"Events: {events} \n")
print(f"Event Indices: {event_ind} \n")

Data: (1280, 8) 

Channels Name: ['Oz', 'O1', 'O2', 'PO3', 'POz', 'PO7', 'PO8', 'PO4'] 

Labels: ['32769' '33024' '32779' ... '32779' '32780' '33026'] 

Events: [[44752     0     1]
 [45264     0     5]
 [45520     0     3]] 

Event Indices: {'32769': 1, '32770': 2, '32779': 3, '32780': 4, '33024': 5, '33025': 6, '33026': 7, '33027': 8} 



In [ ]:
import pandas as pd
# Create a DataFrame for the EEG data
df = pd.DataFrame(data, columns=channels_name)
# Add the labels to the DataFrame
df['Labels'] = labels_new
# Display the first few rows to visualize the table
print(df.head())

             Oz            O1            O2           PO3           POz  \
0  23235.375434   3562.177861    -59.469420 -19892.062483  14099.685808   
1   -472.635590 -12376.284620 -12042.557594 -35832.756058  -9404.871507   
2  -7867.095619 -17384.645801 -17543.478965 -40520.311568 -15822.133913   
3   9833.869524  -5099.699075  -6705.177130 -23482.850197   3978.397834   
4  34502.398036  14659.731967   8177.045280  -4462.192264  25699.535430   

            PO7           PO8           PO4 Labels  
0   8298.748630  16671.916336  22422.587836  32769  
1 -11857.491285  -3945.489097  12022.388204  33024  
2 -15625.571744 -10575.690700   5685.240324  32779  
3    388.770206   6274.217624  19739.154404  32780  
4  23012.205660  26535.639079  32944.072595  33026  


In [ ]:
import numpy as np

# Load the .npy file data (assuming data is in shape [num_trials, num_samples, num_channels])
data1 = np.load('/content/drive/MyDrive/SSVEP-based-EEG-signal-processing-main/SSVEP-based-EEG-signal-processing-main/trials/13hz.npy')[:,:,0]  # 13 Hz trial data
data2 = np.load('/content/drive/MyDrive/SSVEP-based-EEG-signal-processing-main/SSVEP-based-EEG-signal-processing-main/trials/21hz.npy')[:,:,0]  # 21 Hz trial data
data3 = np.load('/content/drive/MyDrive/SSVEP-based-EEG-signal-processing-main/SSVEP-based-EEG-signal-processing-main/trials/17hz.npy')[:,:,0]  # 17 Hz trial data

# Example event information (adjust to match your real event timings and indices)
event_ind = {'32769': 1, '32770': 2, '32779': 3, '32780': 4, '33024': 5,
             '33025': 6, '33026': 7, '33027': 8}  # Define event labels (these map to specific event types)

# Original labels from old dataset (for reference, these will be assigned based on trials)
labels_old = ['32769', '33024', '32779', '32780', '33026', '32779', '32780', '33027',
              '32779', '32780', '33025', '32779', '32780', '33026', '32779', '32780', '33025']

# For this example, we map labels based on the total number of trials (adjust if events follow a different pattern)
labels_new = []

# Example: if we have 1280 trials per data set (change according to the real size of your data)
# Assign labels cyclically from the old labels for each subset of data (data1, data2, data3)
num_trials = data1.shape[0]  # Assuming data1 has 1280 trials
for i in range(num_trials):
    event_idx = i % len(labels_old)  # This will wrap around and assign labels cyclically
    labels_new.append(labels_old[event_idx])

# Convert list to NumPy array
labels_new = np.array(labels_new)

# Assuming data1, data2, data3 are 1280 trials x 8 channels (adjust based on your data)
# Collect all data into one array (e.g., for combined trials) for consistency
all_data = np.concatenate([data1, data2, data3], axis=0)  # Stack data vertically (along rows)

# Channels Names (example for 8 channels)
channels_name = ['Oz', 'O1', 'O2', 'PO3', 'POz', 'PO7', 'PO8', 'PO4']

# Define the sampling frequency (adjust as per your dataset's actual fs)
fs = 256  # Example, adjust based on your data's sample frequency

# Placeholder for event timings (replace with actual event times and values)
events = np.array([[44752, 0, 1], [45264, 0, 5], [45520, 0, 3]])  # Example event array
event_ind = {'32769': 1, '32770': 2, '32779': 3, '32780': 4, '33024': 5,
             '33025': 6, '33026': 7, '33027': 8}  # Map event indices

# Now, let's print the relevant information
print(f"Data Shape: {all_data_microvolts.shape} (trials x channels) \n")
print(f"Channels Names: {channels_name} \n")
print(f"Labels (for {num_trials * 3} trials): {labels_new} \n")
print(f"Events (example): {events} \n")
print(f"Event Indices: {event_ind} \n")

Data Shape: (3840, 8) (trials x channels) 

Channels Names: ['Oz', 'O1', 'O2', 'PO3', 'POz', 'PO7', 'PO8', 'PO4'] 

Labels (for 3840 trials): ['32769' '33024' '32779' ... '32779' '32780' '33026'] 

Events (example): [[44752     0     1]
 [45264     0     5]
 [45520     0     3]] 

Event Indices: {'32769': 1, '32770': 2, '32779': 3, '32780': 4, '33024': 5, '33025': 6, '33026': 7, '33027': 8} 



In [ ]:
import pandas as pd
# Convert data1 into a DataFrame
df = pd.DataFrame(data1)
# Display the DataFrame
print(df)

             0         1         2         3         4         5         6  \
0     0.023235  0.003562 -0.000059 -0.019892  0.014100  0.008299  0.016672   
1    -0.000473 -0.012376 -0.012043 -0.035833 -0.009405 -0.011857 -0.003945   
2    -0.007867 -0.017385 -0.017543 -0.040520 -0.015822 -0.015626 -0.010576   
3     0.009834 -0.005100 -0.006705 -0.023483  0.003978  0.000389  0.006274   
4     0.034502  0.014660  0.008177 -0.004462  0.025700  0.023012  0.026536   
...        ...       ...       ...       ...       ...       ...       ...   
1275  0.030538  0.016974 -0.000877 -0.018435  0.019496  0.014594  0.021700   
1276  0.008767  0.002162 -0.012176 -0.032272 -0.001021 -0.003125 -0.001379   
1277 -0.008751 -0.008540 -0.018911 -0.039619 -0.016264 -0.013054 -0.010442   
1278 -0.003049 -0.007870 -0.017945 -0.037846 -0.010472 -0.008777 -0.005058   
1279  0.020384  0.010702 -0.006497 -0.022702  0.013932  0.010901  0.016865   

             7  
0     0.022423  
1     0.012022  
2     0.0056

In [ ]:
print(data1.shape)
print(data2.shape)
print(data3.shape)

(8,)
(1280, 8)
(1280, 8)


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
import scipy.signal as signal


# Generate the labels for each dataset (e.g., data1 = 13Hz, data2 = 21Hz, data3 = 17Hz)
y1 = np.full(data1.shape[0], 13)  # Label all data1 as 13Hz
y2 = np.full(data2.shape[0], 21)  # Label all data2 as 21Hz
y3 = np.full(data3.shape[0], 17)  # Label all data3 as 17Hz

# Combining data and labels
X = np.vstack([data1, data2, data3])
y = np.concatenate([y1, y2, y3])  #labels/frequencies

#Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Initialize and train a Random Forest classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)
# Predict on the test set
y_pred = clf.predict(X_test)

# Calculate the accuracy
#accuracy = accuracy_score(y_test, y_pred)
# predicted labels back tofrequency
label_to_freq = {
    '13': 13,
    '17': 17,
    '21': 21
}

# Mapping labels to frequencies
predicted_frequencies = [label_to_freq[str(label)] for label in y_pred]

# Output the predicted frequencies for each trial
print("\nPredicted Frequencies for Each Trial:")
for i, freq in enumerate(predicted_frequencies[:20]):  # first 20 predictions
    print(f"Trial {i + 1}: {freq} Hz")


Predicted Frequencies for Each Trial:
Trial 1: 13 Hz
Trial 2: 17 Hz
Trial 3: 13 Hz
Trial 4: 17 Hz
Trial 5: 21 Hz
Trial 6: 17 Hz
Trial 7: 17 Hz
Trial 8: 21 Hz
Trial 9: 17 Hz
Trial 10: 13 Hz
Trial 11: 21 Hz
Trial 12: 17 Hz
Trial 13: 21 Hz
Trial 14: 21 Hz
Trial 15: 17 Hz
Trial 16: 13 Hz
Trial 17: 17 Hz
Trial 18: 21 Hz
Trial 19: 17 Hz
Trial 20: 17 Hz
